<a href="https://colab.research.google.com/github/jelade/Jaleel/blob/master/Prediction_Car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 77 kB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score , accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
#xgb.XGBRegressor

In [ ]:
test = pd.read_csv('/content/Test(1).csv')
train = pd.read_csv('/content/Train(1).csv')
sub = pd.read_csv('/content/SampleSubmission(1).csv')
var_def = pd.read_csv('/content/VariableDefinitions(1).csv')

In [ ]:
train.shape, test.shape

((7205, 9), (2061, 8))

In [ ]:
var_def

,VehicleID,This is the unique identifier of the car.
0,Location,This is the location in Nigeria where the sell...
1,Maker,This is the manufacturer of the car. It is the...
2,Model,This is the the name of the car product within...
3,Year,This is the year the car was manufactured.
4,Colour,This is the colour of the car.
5,Amount (Million Naira),This is the selling price of the car. It is th...
6,Type,"This is the nature of previous use of the car,..."
7,Distance,This is the mileage of the car. It is how much...


In [ ]:
train[['Distance','Year']]=train[['Distance','Year']].apply(lambda x: x.str.replace(',',''))


In [ ]:
test["Year"] = test["Year"].astype(str)
test['Year']=test['Year'].apply(lambda x: x.replace(',',''))

In [ ]:
train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,2012,Silver,3.5,Nigerian Used,125000
2,VHL19499,Lagos,Lexus,RX 350,2010,Red,9.2,Foreign Used,110852
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.8,Foreign Used,30000
4,VHL12170,Ibadan,Toyota,Highlander,2002,Red,2.6,Nigerian Used,125206


In [ ]:
test.isna().sum()

VehicleID      0
Location       0
Maker          0
Model          0
Year           0
Colour         0
Type          54
Distance     676
dtype: int64

In [ ]:
test.head()

,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,Abuja,BMW,323i,2008,White,Foreign Used,30524.0
1,VHL17149,Lagos,Toyota,Camry,2013,White,Foreign Used,NaN
2,VHL10927,Lagos,Toyota,Highlander Limited V6,2005,Gold,Foreign Used,NaN
3,VHL12909,Lagos,Toyota,Camry,2011,Gray,Foreign Used,166839.0
4,VHL12348,Lagos,Lexus,ES 350 FWD,2013,Red,Foreign Used,88862.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7184 non-null   object 
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7188 non-null   float64
 7   Type                    7008 non-null   object 
 8   Distance                4845 non-null   object 
dtypes: float64(1), object(8)
memory usage: 506.7+ KB


In [ ]:
mean1 = train[train['Type']== "Foreign Used"]['Distance'].dropna()
mean1.str.replace(',','').astype('int').mean()

88183.3909375

In [ ]:
mean2 = train[train['Type']== "Nigerian Used"]['Distance'].dropna()
mean2.str.replace(',','').astype('int').mean()

144847.418429003

In [ ]:
mean3 = train[train['Type']== "Brand New"]['Distance'].dropna()
mean3.str.replace(',','').astype('int').mean()

4794.769841269841

In [ ]:
mean4 = test[test['Type']== "Foreign Used"]['Distance'].dropna()
mean4.mean()

90903.7178924259

In [ ]:
mean5 = test[test['Type']== "Nigerian Used"]['Distance'].dropna()
mean5.mean()

137584.7265625

In [ ]:
mean6 = test[test['Type']== "Brand New"]['Distance'].dropna()
mean6.mean()

3201.6388888888887

In [ ]:
for i in train["Type"]:
  if i == "Foreign Used":
    train['Distance'].fillna(88183, inplace = True)
  if i == "Brand New":
    train['Distance'].fillna(4795, inplace = True)
  else:
    train['Distance'].fillna(144847, inplace = True)

In [ ]:
for i in test["Type"]:
  if i == "Foreign Used":
    test['Distance'].fillna(90904, inplace = True)
  if i == "Brand New":
    test['Distance'].fillna(3202, inplace = True)
  else:
    test['Distance'].fillna(137585, inplace = True)

In [ ]:
test.isnull().sum()

VehicleID     0
Location      0
Maker         0
Model         0
Year          0
Colour        0
Type         54
Distance      0
dtype: int64

In [ ]:
train.isnull().sum()

VehicleID                   0
Location                    0
Maker                       0
Model                       0
Year                       21
Colour                      0
Amount (Million Naira)     17
Type                      197
Distance                    0
dtype: int64

In [ ]:
train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.2,Nigerian Used,144847
1,VHL18827,Ibadan,Hyundai,Sonata,2012,Silver,3.5,Nigerian Used,125000
2,VHL19499,Lagos,Lexus,RX 350,2010,Red,9.2,Foreign Used,110852
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.8,Foreign Used,30000
4,VHL12170,Ibadan,Toyota,Highlander,2002,Red,2.6,Nigerian Used,125206


In [ ]:
train['Distance'] = pd.to_numeric(train['Distance'], downcast="float")


In [ ]:
train['Year'] = pd.to_numeric(train['Year'])

In [ ]:
yearna = train[train['Year'].isna() == True]

In [ ]:
train['Year'] = train['Year'].fillna(method='ffill')

In [ ]:
train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011.0,Silver,2.2,Nigerian Used,144847.0
1,VHL18827,Ibadan,Hyundai,Sonata,2012.0,Silver,3.5,Nigerian Used,125000.0
2,VHL19499,Lagos,Lexus,RX 350,2010.0,Red,9.2,Foreign Used,110852.0
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017.0,Blue,22.8,Foreign Used,30000.0
4,VHL12170,Ibadan,Toyota,Highlander,2002.0,Red,2.6,Nigerian Used,125206.0


In [ ]:
train['Year'] = train['Year'].apply(np.int64)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7205 non-null   int64  
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7188 non-null   float64
 7   Type                    7008 non-null   object 
 8   Distance                7205 non-null   float32
dtypes: float32(1), float64(1), int64(1), object(6)
memory usage: 478.6+ KB


In [ ]:
for i in train["Amount (Million Naira)"]:
  if i <= 1.30:
    train['Type'].fillna('Nigerian Used', inplace = True)
  if i <= 300:
    train['Type'].fillna('Foreign Used', inplace = True)
  else:
    train['Type'].fillna('Brand New', inplace = True)

In [ ]:
amount22 = train[train['Amount (Million Naira)'].isna() == True]

In [ ]:
amount22.index

Int64Index([ 288,  751, 1549, 2006, 2250, 3094, 3207, 3550, 3727, 4240, 4400,
            4701, 4752, 5026, 6166, 6275, 6373],
           dtype='int64')

In [ ]:
gg = train.groupby(['Year','Maker'])['Amount (Million Naira)'].mean()
gg.loc[2012,'Honda']

3.7661904761904763

In [ ]:
train.loc[288,'Amount (Million Naira)'] = 8.69
train.loc[751,'Amount (Million Naira)'] = 7.25
train.loc[3550,'Amount (Million Naira)'] = 3.29
train.loc[3727,'Amount (Million Naira)'] = 16.76
train.loc[751,'Amount (Million Naira)'] = 4.12
train.loc[6275,'Amount (Million Naira)'] = 2.63
train.loc[2006,'Amount (Million Naira)'] = 1.72
train.loc[2250,'Amount (Million Naira)'] = 4.32
train.loc[3094,'Amount (Million Naira)'] = 6.57
train.loc[4240,'Amount (Million Naira)'] = 6.96
train.loc[3207,'Amount (Million Naira)'] = 5.01
train.loc[4400,'Amount (Million Naira)'] = 4.68
train.loc[5026,'Amount (Million Naira)'] = 2.07
train.loc[6166,'Amount (Million Naira)'] =13.99
train.loc[6373,'Amount (Million Naira)'] =43.8
train.loc[1549,'Amount (Million Naira)'] = 7.25
train.loc[4701,'Amount (Million Naira)'] = 7.9
train.loc[4752,'Amount (Million Naira)'] = 3.77

In [ ]:
train.isnull().sum()

VehicleID                 0
Location                  0
Maker                     0
Model                     0
Year                      0
Colour                    0
Amount (Million Naira)    0
Type                      0
Distance                  0
dtype: int64

In [ ]:
train = train.drop('VehicleID', axis = 1)


In [ ]:
train.head()

,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.2,Nigerian Used,144847.0
1,Ibadan,Hyundai,Sonata,2012,Silver,3.5,Nigerian Used,125000.0
2,Lagos,Lexus,RX 350,2010,Red,9.2,Foreign Used,110852.0
3,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.8,Foreign Used,30000.0
4,Ibadan,Toyota,Highlander,2002,Red,2.6,Nigerian Used,125206.0


In [ ]:
df1_tr = pd.get_dummies(train)
df1_tr.head()

,Year,Amount (Million Naira),Distance,Location_Abuja,Location_Ibadan,Location_Lagos,Maker_Acura,Maker_Audi,Maker_BAW,Maker_BMW,...,Colour_Purple,Colour_Red,Colour_Silver,Colour_Teal,Colour_Violet,Colour_White,Colour_Yellow,Type_Brand New,Type_Foreign Used,Type_Nigerian Used
0,2011,2.2,144847.0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,2012,3.5,125000.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,2010,9.2,110852.0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,2017,22.8,30000.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2002,2.6,125206.0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
y1 = df1_tr['Amount (Million Naira)']
x1 = df1_tr.drop('Amount (Million Naira)', axis = 1)


#x_test1 = test2[['Location','Maker','Model','Year','Colour','Type','Distance']]

In [ ]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1, test_size = 0.2, random_state = 42)

In [ ]:
x1_train.shape

(5764, 1307)

In [ ]:
model2 = LinearRegression().fit(x1_train,y1_train)
pred2 = model2.predict(x1_test)
score2 = np.sqrt(mean_squared_error(y1_test,pred2))
score2

24335.417811086434

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
train1 = train.copy()

In [ ]:
for i in train1[['Location','Maker', 'Year','Model','Colour','Type']]:
  train1[i] = train1[i].astype("category").cat.codes
train1.head()

,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,0,17,117,21,16,2.2,2,144847.0
1,1,19,1049,22,16,3.5,2,125000.0
2,2,29,908,20,15,9.2,1,110852.0
3,0,34,508,27,2,22.8,1,30000.0
4,1,52,569,12,15,2.6,2,125206.0


In [ ]:
test2 = test.copy()

In [ ]:
for i in test2[['Location','Maker', 'Year','Model','Colour','Type']]:
  test2[i] = test2[i].astype("category").cat.codes
test2.head()

,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,0,2,8,14,16,1,30524.0
1,VHL17149,2,37,123,19,16,1,90904.0
2,VHL10927,2,37,272,11,7,1,90904.0
3,VHL12909,2,37,123,17,8,1,166839.0
4,VHL12348,2,20,192,19,12,1,88862.0


In [ ]:
test2['Distance'] = test2['Distance'].astype('float32')

In [ ]:

#test2['Distance'] = test2['Distance'].astype('int')

In [ ]:
y = train1['Amount (Million Naira)']
x = train1[['Location','Maker','Model','Year','Colour','Type','Distance']]


x_test1 = test2[['Location','Maker','Model','Year','Colour','Type','Distance']]

In [ ]:
x_test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Location  2061 non-null   int8   
 1   Maker     2061 non-null   int8   
 2   Model     2061 non-null   int16  
 3   Year      2061 non-null   int8   
 4   Colour    2061 non-null   int8   
 5   Type      2061 non-null   int8   
 6   Distance  2061 non-null   float32
dtypes: float32(1), int16(1), int8(5)
memory usage: 22.3 KB


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Location  7205 non-null   int8   
 1   Maker     7205 non-null   int8   
 2   Model     7205 non-null   int16  
 3   Year      7205 non-null   int8   
 4   Colour    7205 non-null   int8   
 5   Type      7205 non-null   int8   
 6   Distance  7205 non-null   float32
dtypes: float32(1), int16(1), int8(5)
memory usage: 77.5 KB


In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
model1 = AdaBoostRegressor(random_state=0, n_estimators=100)
model1.fit(x_train,y_train)
pred1 = model1.predict(x_test)
pred2= model1.predict(x_test1)

In [ ]:
#def to_int(x):
#  return int(x)
#x_test['Year']=x_test['Year'].apply(lambda x: int(x))

In [ ]:
score1 = np.sqrt(mean_squared_error(y_test,pred1))
score1

19.900518962302204

In [ ]:
sub['Amount (Million Naira)'] = pred2

In [ ]:
sub.to_csv('submission2.csv')

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
test1 = test.copy()

In [ ]:
for i in test1[['Location','Maker','Model','Colour']]:
  test1[i] = test1[i].astype("category").cat.codes
test1.head()

In [ ]:
test.isnull().sum()

In [ ]:
tt = test1[test1['Type'].isna() == True].reset_index()
tt = tt.dropna(axis=1)

In [ ]:
ttr = test1.dropna()
ttr.shape

In [ ]:
tt = tt.drop(['index','VehicleID'], axis = 1)
ttr = ttr.drop('VehicleID', axis = 1)

In [ ]:
ttr.head()

In [ ]:
ttr.nuniqu

In [ ]:
ttr['Type'] = ttr['Type'].astype("category").cat.codes


In [ ]:
tt = tt.drop(['index','VehicleID'], axis = 1)

In [ ]:
x = ttr['Type']
y = ttr[['Location','Maker','Model','Year','Colour','Distance']]

In [ ]:
y.head()

In [ ]:
ttr.mean()

In [ ]:
y.info()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import AffinityPropagation

In [ ]:
scaler = StandardScaler()
# transform data
scaled = scaler.fit_transform(y)
print(scaled)

In [ ]:
Kmean = AffinityPropagation()
Kmean.fit(scaled)